# Process Amazon reviews

### Install dependencie

In [ ]:
!pip install confluent-kafka

### Import libraries

In [ ]:
from confluent_kafka import Producer, Consumer, KafkaError # to produce and consume data from Apache Kafka topics
import boto3 # to programmatically create, configure, and manage AWS resources
import json # to work with social media messages that are represented as JSON objects
import re # for helper functionality to clean HTML tags from social media messages


## Prepare models

### Use a JumpStart models to predict if a message is positive or negative
The model you can use is Text Classification by HuggingFace

In [ ]:
# Define a mapping dictionary to map model labels to negative/positive label
label_mapping = {'LABEL_0': 'negative', 'LABEL_1': 'positive'}

def get_prediction(text):
    endpoint_name = '' # TODO: Set endpoint name of your model for sentiment analysis
    client = boto3.client('runtime.sagemaker')
    query_response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-text', Body=text, Accept='application/json;verbose')
    model_predictions = json.loads(query_response['Body'].read())
    probabilities, labels, predicted_label = model_predictions['probabilities'], model_predictions['labels'], model_predictions['predicted_label']
    # Map the predicted_label to your the label using the mapping dictionary
    predicted_label = label_mapping.get(predicted_label, predicted_label)
    return probabilities, labels, predicted_label


### Use a JupmStart zero-shot model to predict categories of the product
You can use Zero-Shot Text Classification by HuggingFace

In [ ]:
categories = [
  "Snacks",
  "Coffee",
  "Beverages",
  "Condiments",
  "Canned Goods",
  "Bakery and Baked Goods",
  "Dairy Products",
  "Sweets and Desserts",
  "Breakfast Foods",
  "Grains and Pasta",
  "Protein Products",
  "Frozen Foods",
  "Health and Nutrition",
  "Baby and Toddler Food",
  "Pet Supplies",
  "Fruits and Vegetables",
  "Herbs and Spices",
  "Beverage Accessories",
  "Cooking Oils and Fats",
  "Ethnic and Specialty Foods",
  "Dietary Supplements"
];
def get_categories(text):
    endpoint_name = '' # TODO: Set endpoint name of your model for classification
    client = boto3.client('runtime.sagemaker')
    
    model_input = {"sequences": text, "candidate_labels": categories, "multi_class": True}
    query_response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps(model_input),
        Accept='application/json;verbose'
    )
    
    model_predictions = json.loads(query_response['Body'].read())
    
    # Filter categories with probabilities over 60%
    filtered_categories = [
        label_mapping.get(label, label)
        for label, score in zip(model_predictions['labels'], model_predictions['scores'])
        if score > 0.6
    ]
    
    return filtered_categories


### Test endpoint for classifications

In [ ]:
print(get_categories("very delicious coffee for my cat who likes to snack on tiramisu"))


## Apache Kafka workflow

### Define Apache Kafka connection properties

In [ ]:
# TODO: Load Apache Kafka certificates into certificates folder
apache_kafka_ssl_config = {
    'ssl.ca.location': 'certificates/ca.pem', 
    'ssl.certificate.location': 'certificates/service.cert',
    'ssl.key.location': 'certificates/service.key',
    'security.protocol': 'ssl',
}

apache_kafka_uri = '' # TODO: Set URI for Apache Kafka

apache_kafka_input_topic_name = 'reviews'
apache_kafka_enriched_output_topic_name = 'reviews_enriched'
apache_kafka_processing_errors_topic_name = 'reviews_processing_errors'


### Create Apache Kafka Consumer

In [ ]:
consumer = Consumer({'bootstrap.servers': apache_kafka_uri, 'group.id': 'mygroup10', 'auto.offset.reset': 'earliest', **apache_kafka_ssl_config})
consumer.subscribe([apache_kafka_input_topic_name])

CLEANR = re.compile('<.*?>') 

def get_json_body(message):    
    decoded_message = message.value().decode('utf-8') # Decode from binary 
    json_message = json.loads(decoded_message)  # Parse JSON message
    return json_message

def get_clean_content(json_object):    
    content = json_object.get("Text", "")  # Retrieve 'content' property    
    only_text = re.sub(CLEANR, '', content)
    return only_text


### Create Apache Kafka Producer

In [ ]:
producer = Producer({
    'bootstrap.servers': apache_kafka_uri, 
    **apache_kafka_ssl_config 
})

# Send a message to a Kafka topic
def send_message(message, topic_name):
    producer.produce(topic_name, json.dumps(message).encode('utf-8'))
    producer.flush()
    
def send_enriched_data(message, probabilities, predicted_label, key_categories):
    message['probabilities'] = probabilities
    message['sentiment_predition'] = predicted_label
    message['categories'] = key_categories
    send_message(message, apache_kafka_enriched_output_topic_name)
    
def report_processing_error(message, error_code, error_message):
    message['processing_error_code'] = error_code
    message['processing_error_message'] = error_message
    send_message(message, apache_kafka_processing_errors_topic_name)
    

### Read messages from Apache Kafka **input topic** and push processed data back to **output topic**

In [ ]:
print(f"Processing messages")
while True:
    message = consumer.poll(1.0)  # Poll for messages, with a timeout of 1 second

    if message is None:
        continue

    if message.error():
        if message.error().code() == KafkaError._PARTITION_EOF:
            # End of partition event
            print(f"Reached end of partition for topic {message.topic()} [{message.partition()}]")
        else:
            print(f"Error while consuming message: {message.error()}")
    else:
        # Process the message
        json_body = get_json_body(message)
        content_property = get_clean_content(json_body)
        print(f"Process the messagee: {content_property}")
        if content_property == "":
            continue

        try:
            probabilities, labels, predicted_label = get_prediction(content_property)
            key_categories = get_categories(content_property)
            print(f"Inference:\n"
                  f"Input text: '{content_property}'\n"
                  f"Model prediction: {probabilities}\n"
                  f"Predicted label: {predicted_label}\n"
                  f"Predicted key_categories: {key_categories}\n")

            send_enriched_data(json_body, probabilities, predicted_label, key_categories)
            

        except Exception as e:
            print(f"An error occurred: {e}")
            response = getattr(e, "response", {})
            error_code = response.get("Error", {}).get("Code", "Unknown")
            error_message = response.get("Error", {}).get("Message", "Unknown")
            report_processing_error(json_body, error_code, error_message)
            

# Close the consumer
consumer.close()